## 3-3 : Static Downscaling with Full Workflow
<em>Created by Tomás Cuevas López in 2024, modified by Casey Dietrich in 2024.</em><br>
<br>
This example will show how to use the full workflow for static downscaling, including the pre-processing of the ADCIRC mesh.<br>
<br>
This example is a simple combination of the previous two examples. Previously, we used two steps to (a) pre-process the ADCIRC mesh to create a raster DEM with the mesh element sizes, and (b) downscale the ADCIRC maximum water levels. In this example, we do everything as one step. This can take longer, especially if the mesh and/or DEM are large.<br>
<br>
We use files from a simulation of Hurricane Ida, which devastated the south Louisiana coast in 2021. To minimize the file sizes in this repository, we created a small mesh that includes Grand Isle and Barataria Bay. Then we simulated Ida by using the NHC best-track storm parameters in a vortex wind model. The simulated storm effects are meant to be useful as an example, but they should not be relied on for any decision making.

## Preliminaries

In [1]:
import os
import glob
import shutil
from pathlib import Path
from kalpana.downscaling import runStatic

## Full Workflow for Static Downscaling

The following cell defines our parameters for the mesh pre-processing. All of these parameters are repeated from Example 3-1.

In [2]:
######## For more detail about the meshRepLen2raster inputs, please read the function's docstring.

## Use a Python function to get the path for the current working directory.
cwd = Path(os.getcwd())

## Path of the ADCIRC mesh file (fort.14).
fort14 = str(cwd.parent.parent/'adds'/'ida'/'fort.14')

## Coordinate system of the ADCIRC input.
## Default is 4326 because ADCIRC uses latitude and longitude.
epsgIn = 4326

## Coordinate system of the new raster DEM with the mesh element sizes.
## We use a pseudo-Mercator (3857), but UTM projections (e.g. 6344, 6345) can also work fine.
epsgOut = 3857

## Path of the new raster with the mesh element sizes.
## Kalpana will create both a raster DEM (geoTIFF) and a shapefile.
pathOut = os.path.join(cwd, 'CUDEM_merged_10m_crs3857_mesh.shp') 

## Version of GRASS GIS.
grassVer = 8.3 ## versions 8.2 and 8.3 work

## Path to the folder with the input DEM with the ground surface elevations.
pathRasFiles = str(cwd.parent.parent/'adds'/'ida')

## Name(s) of input DEM(s) with the ground surface elevations.
rasterFiles = 'CUDEM_merged_10m_crs3857.tif'

## Bounding box for the downscaling raster.
## In this example, we will use the same downscaling raster bounding box as the input DEM. 
subDomain = os.path.join(pathRasFiles, rasterFiles) 

## Name of the GRASS location.
nameGrassLocation = None

## True for creating a new location and loading DEMs, false to use an existing location with DEMs already imported.
createGrassLocation = True

## Method for assigning the crs to the grass location.
## Two options 'from_epsg' or 'from_raster' otherwise an error will be thrown.
## In this example, we use the projection from the input DEM.
createLocMethod = 'from_raster'

The following cell defines our parameters for the static downscaling. Note that a few parameters have changed from Example 3-2: <code>exportMesh</code> is True, <code>nameGrassLocation</code> is None, and <code>createGrassLocation</code> is True.

In [3]:
######## For more detail about the runStatic inputs, please read the function's docstring.

## Full path of the ADCIRC maxele file.
cwd = Path(os.getcwd())
ncFile = str(cwd.parent.parent/'adds'/'ida'/'maxele.63.nc') 

## Contour levels to use in the downscaling.
## From 0 to 6 m (included) every 0.5 m.
levels = [0, 6, 0.5]

## Projection to use for the output downscaled DEM.
## In this example, we use pseudo-Mercator (3857), but a user could also use UTM (e.g. 6344, 6345).
epsgOut = 3857

## Full path for the output downscaled DEM.
## Same path is used for saving rasters and the grass location.
pathOut = os.path.join(os.getcwd(), 'maxele_Ida.shp')

## Version of GRASS GIS.
grassVer = 8.3 ## versions 8.2 and 8.3 work

## Path to the folder with the input DEM with the ground surface elevations.
pathRasFiles = str(cwd.parent.parent/'adds'/'ida')

## Name(s) of input DEM(s) with the ground surface elevations.
rasterFiles = 'CUDEM_merged_10m_crs3857.tif'

## Full path of the raster DEM with the mesh element sizes.
meshFile = r'CUDEM_merged_10m_crs3857_mesh.tif'

## Coordinate system of the ADCIRC input.
## Default is 4326 because ADCIRC uses latitude and longitude.
epsgIn = 4326

## Vertical unit of the ADCIRC maximum water levels.
vUnitIn = 'm'

## Vertical unit of the downscaled water levels.
vUnitOut = 'm'

## Name of the maxele variable to downscale.
var = 'zeta_max'

## Contours type.
conType = 'polygon' ## Always 'polygon' for downscaling.

## Bounding box for the downscaling raster.
## Full path of file (kml, kmz, shp, gpkg or tif) to crop the domain.
## In this example, we will use the same downscaling raster bounding box as the input DEM. 
subDomain = os.path.join(pathRasFiles, rasterFiles)

## Projection (epsg code or crs) of the subDomain.
## In this example, as we are using the downscaling dem bounding box as the subdomain, so the same epsg code must be specified.
epsgSubDom = 3857

## Boolean for exporting the mesh as a shape file from maxele.
## Necessary in this example because the mesh was not exported as preprocess.
exportMesh = True

## Full path of pickle file with vertical datum differences for all mesh nodes preprocess step.
dzFile = r'../../adds/dzDatumsNOAA/dzDaums_noaaTideGauges_msl_navd88.csv'
## Threshold to apply the vertical datum difference.
zeroDif = -20
## Threshold to define the percentage of the dz given by the spatial interpolation to be applied.
maxDif = -5
## Only tide stations closed than this threshold are used to interpolate the vertical datum difference.
distThreshold = 0.5

## Number of points to query for the inverse distance-weighted interpolation.
k = 7

## Full path of the grass location if a existing location will be used.
## If None, then a new location called 'grassLoc' is created.
## Path can't be relative.
nameGrassLocation = None

## Boolean for creating grass location.
## In this example it will be created.
createGrassLocation = True

## Method for assigning the crs to the grass location.
## Two options 'from_epsg' or 'from_raster' otherwise an error will be thrown.
## In this example, we use the projection from the input DEM.
createLocMethod = 'from_raster'

## Variable to downscale, can be 'zMax', 'zMean' and 'zMin'.
## With 'zMean', the mean value of each contour is used.
attrCol = 'zMean'

## How many times the representative length the results are grown in the downscaling.
repLenGrowing = 1.0 

## Remove wet cells with water level below the ground surface.
compAdcirc2dem = True

## Transform the water level to water depth.
floodDepth = False

## Export downscaled results as shape files.
## Slows down the process a couple of minutes.
ras2vec = False

## Boolean for exporing raw maxele as a DEM. Useful for debugging.
exportOrg = False

## Full path of the shapefile with levees.
leveesFile = None

## Boolean for reprojecting the downscaled dem back to lat/lon.
finalOutToLatLon = False

Now, let's call the <code>runStatic</code> function with the parameters we defined. This may take a few minutes, especially because now we must pre-process the mesh before doing the downscaling.

In [4]:
#################### calling downscaling
runStatic(ncFile, levels, epsgOut, pathOut, grassVer, pathRasFiles, rasterFiles, meshFile, epsgIn=epsgIn, 
            vUnitIn=vUnitIn, vUnitOut=vUnitOut, var=var, conType =conType, subDomain=subDomain, epsgSubDom=epsgSubDom, 
            exportMesh= exportMesh, dzFile=dzFile, zeroDif=zeroDif, maxDif=maxDif, distThreshold=distThreshold, k=k, 
            nameGrassLocation=nameGrassLocation, createGrassLocation=createGrassLocation, createLocMethod=createLocMethod, 
            attrCol=attrCol, repLenGrowing=repLenGrowing, compAdcirc2dem=compAdcirc2dem, floodDepth=floodDepth, 
            ras2vec=ras2vec, exportOrg=exportOrg, leveesFile = leveesFile, finalOutToLatLon=finalOutToLatLon)

2024-07-25 10:34:04.262 | INFO     | kalpana.export:nc2shp:817 - Start exporting adcirc to shape
2024-07-25 10:34:05.182 | INFO     | kalpana.export:filledContours2gpd:311 - Begin computing contours using Dask
2024-07-25 10:34:05.260 | INFO     | kalpana.export:filledContours2gpd:313 - Finnished computing contours using Dask
2024-07-25 10:34:05.269 | INFO     | kalpana.export:nc2shp:847 -     Ready with the contours extraction: 0.016 min
2024-07-25 10:34:05.325 | INFO     | kalpana.export:nc2shp:855 -     Cliping contours based on mask: 0.001 min
2024-07-25 10:34:05.333 | INFO     | kalpana.export:nc2shp:873 -     Changing CRS: 0.000 min
2024-07-25 10:34:05.405 | INFO     | kalpana.export:nc2shp:884 -     Saving file: 0.001 min
2024-07-25 10:34:05.405 | INFO     | kalpana.export:nc2shp:888 -     Exporting mesh
2024-07-25 10:35:12.968 | INFO     | kalpana.export:nc2shp:896 -     Mesh exported: 1.126 min
2024-07-25 10:35:12.969 | INFO     | kalpana.export:nc2shp:897 - Ready with exportin

The outputs of this function are the same as for the previous Examples 3-1 and 3-2. Please see those notebooks for visualizations of: the new raster DEM with the mesh element sizes, and the downscaled water surface.

## Housekeeping

The following cell is optional. We include it to keep clean the repository -- it will delete the shapefile and geoTIFF that we just created. For most users, this cell is not necessary.

In [6]:
## clean
for f in glob.glob("CUDEM_merged_10m_crs3857_mesh*"):
    os.remove(f)

for f in glob.glob("downscaling_dem*"):
    os.remove(f)

for f in glob.glob("maxele_Ida*"):
    os.remove(f)

shutil.rmtree('grassLoc')

Good luck with Kalpana! See you in future examples!